<a href="https://colab.research.google.com/github/vydra-v-getrax/training-projects/blob/master/lenta_ru_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Импорты и загрузка данных

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/lenta-ru-news_19-21_raw.zip' -d '/content/drive/MyDrive/lenta_news'

Archive:  /content/drive/MyDrive/lenta-ru-news_19-21_raw.zip
  inflating: /content/drive/MyDrive/lenta_news/lenta-ru-news_19-21_raw.csv  


In [3]:
filepath = '/content/drive/MyDrive/lenta_news/lenta-ru-news_19-21_raw.csv'
folderpath = '/content/drive/MyDrive/lenta_news'

In [1]:
import pandas as pd
import numpy as np
import os
from time import time
from tqdm.auto import tqdm
tqdm.pandas()

!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from string import punctuation
import re
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")

# библиотека для токенизации текстов
!pip install razdel
from razdel import tokenize, sentenize

# для извлечения темы
import gensim
from gensim import matutils
from gensim.models.ldamodel import LdaModel
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csc

from gensim.summarization import keywords as kw


import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [104]:
data = pd.read_csv(filepath, index_col=0)

In [106]:
data.head()

,url,title,text
0,https://lenta.ru/news/2019/01/01/film/,Назван главный российский фильм 2018 года,Президент Гильдии киноведов и кинокритиков России Кирилл Разлогов назвал главный российский филь...
1,https://lenta.ru/news/2019/01/01/zelenskiy/,Украинский комик поборется за президентский пост,Ведущий украинского комедийного шоу «Вечерний квартал» Владимир Зеленский примет участие в прези...
2,https://lenta.ru/news/2019/01/01/xvii_century/,Киевских политиков обвинили в попытке вернуть страну в XVII век,Бывший президент Украины Виктор Ющенко назвал обещания киевских политиков снизить цены на газ по...
3,https://lenta.ru/news/2019/01/01/klimkin/,Климкин погрустнел из-за советских фильмов,Министр иностранных дел Украины Павел Климкин расстроился из-за большого количества советских фи...
4,https://lenta.ru/news/2019/01/01/otsenka/,Уточнено число пропавших после обрушения дома в Магнитогорске,ГУ МЧС по Челябинской области уточнило число пропавших людей после обрушения подъезда дома в Маг...


#Подготовка данных

In [ ]:
data['dt'] = data.url.apply(lambda x: x.split('/')[5] + '.' + x.split('/')[4])
data.dt.unique()

In [ ]:
data.fillna('', inplace=True)

In [108]:
# Для загрузки после промежуточного сохранения:

# data = pd.read_pickle(os.path.join(folderpath, 'data.pkl'))

In [ ]:
# функция для лемматизации и удаления пунктуации и стоп-слов
morph = MorphAnalyzer()
russian_stopwords = stopwords.words("russian")

def preprocess(text):
    tokens = tokenize(text.lower())
    tokens = [token.text for token in tokens 
              if token.text not in punctuation + '«»...–—!?' 
              and token.text not in russian_stopwords]
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    return ' '.join(lemmas)

In [ ]:
data['lemmas'] = data.title.progress_apply(preprocess)

  0%|          | 0/29650 [00:00<?, ?it/s]

In [ ]:
data['text_lemmas'] = data.text.progress_apply(preprocess)

  0%|          | 0/29650 [00:00<?, ?it/s]

In [49]:
# промежуточное сохранение
# data.to_pickle(os.path.join(folderpath, 'data.pkl'))

#Выделение тем

Будем выделять ключевые слова из каждого текста методом TextRank. Для каждого слова строится граф совместной встречаемости с другими словами, узлы с наибольшими весами становятся ключевыми словами. Можно воспользоваться реализацией библиоткеи gensim.
Будем оставлять только существительные, глаголы и прилагательные, как наиболее "важные" части речи. 

Сначала я пробовала выделять слова только из заголовков, но получается мало информативно. Обработка всех текстов заняла около 20 минут.

In [20]:
data['kw_textrank'] = data.text_lemmas.apply(lambda x: kw(x, pos_filter=('NN', 'VV', 'JJ')).split('\n'))

In [112]:
print("Пример выделенных ключевых слов:")
data.kw_textrank[0]
#получается довольно чисто и красиво

Пример выделенных ключевых слов:


['год',
 'россиискии фильм',
 'разлогов',
 'стать картина',
 'премия оскар',
 'гильдия',
 'главныи',
 'хорошии',
 'кинокритик',
 'движение',
 'тренер добавить',
 'билборд граница',
 'составить']

Сгруппируем тексты по месяцу новости

In [7]:
gr = data.groupby('dt').agg({'kw_textrank': list, 'kw_titles': lambda x: ' '.join(x)})

In [29]:
gr['kw_text'] = gr.kw_textrank.apply(lambda x: [' '.join(s) for s in x])

In [114]:
gr.head()

,kw_textrank,kw_text
dt,,
01.2019,"[[год, россиискии фильм, разлогов, стать картина, премия оскар, гильдия, главныи, хорошии, кинок...",[год россиискии фильм разлогов стать картина премия оскар гильдия главныи хорошии кинокритик дви...
01.2020,"[[мрот, размер, рубль, год, декабрь, путин, которыи, налог, закон, ниже], [маска, рогозин заявит...","[мрот размер рубль год декабрь путин которыи налог закон ниже, маска рогозин заявить роскосмос с..."
01.2021,"[[должныи, школа, правило, год, время, также, ребенок, новыи, санитарныи требование организация,...",[должныи школа правило год время также ребенок новыи санитарныи требование организация образоват...
02.2019,"[[американскии истребитель, самолет, раннии, год налетать, проектныи, списать, современныи требо...",[американскии истребитель самолет раннии год налетать проектныи списать современныи требование э...
02.2020,"[[закон, сми, реалия, иноагент, россия, юридическии лицо, иностранныи агент мочь признать, росси...",[закон сми реалия иноагент россия юридическии лицо иностранныи агент мочь признать россиискии ст...


Теперь на основе выделенных ключевых слов попробуем определить главные темы для каждого месяца с помощью LDA. Мне не понравились результаты выделения тем на исходных текстах, вижу следующие причины:


*   большие объемы
*   много однообразных новостей на похожие темы (политика, одни и те же имена и термины, много похожих статей про коронавирус и тд.), много шума

Поэтому я решила запускать тематическое моделирование уже поверх выделенных ключевых слов. У этого, конечно, тоже есть минусы: теряется много информации из исходных текстов.

Просмотрев  самые частые ключевые слова из пары текстов, я решила дополнительно удалить локальные стоп-слова.

Пользоваться автоматически собранным списком самых частотных слов неудобно, так как могут удалиться важные, например "президент", "коронавирус" и др. 

Конечно, для построения более сложных моделей "ручной" способ не подходит, но здесь это было намного быстрее и осмысленнее.



In [119]:
add_stopwords = ['год', 'которыи', 'также', 'свои', 'россия', 'это', 'россиискии', 'страна', 'сша', 
                 'такои', 'человек', 'самыи', 'назвать', 'рассказать', 'раскрыть', 'оценить', 
                 'сообщать', 'сказать', 'заявить','стать', 'однако', 'несколько', 'процент', 'один', 
                 'время', 'ранее', 'мужчина', 'женщина', 'январь', 'февраль', 'март', 'апрель', 'май', 
                 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']

In [123]:
# получим эмбеддинги слов с помощью Count Vectorizer

def get_vector(corpus, vector_type):
    X = vector_type.fit_transform(corpus)
    vocab = vector_type.get_feature_names()
    
    return X, vocab

Обучим LDA, взяв 15 эпох и 10 топиков в каждом тексте. Я выбрала эти значения приблизительно, из экономии времени. Вообще нужно подбирать лучшее значение, например, с помощью GridSearch. 

In [122]:
N_TOPICS = 10
N_PASSES = 15

def fit_lda(X, vocab, num_topics=N_TOPICS, passes=N_PASSES):
    print('fitting lda...')
    return LdaModel( matutils.Sparse2Corpus(X.T), num_topics=num_topics,
                    passes=passes, 
                    id2word=dict([(i, s) for i, s in enumerate(vocab)]))

In [ ]:
# функция для получения ключевых слов каждой выделенной темы

def print_topics(lda, vocab, n_words=5, n_topics=5):
    """ Print the top words for each topic. """
    topics = lda.print_topics(num_topics=n_topics, num_words=n_words)
    output = [re.findall('\"(.+?)\"', topic[1]) for topic in topics]
    return output

In [120]:
def lda_month(month):
  count_vectorizer = CountVectorizer(stop_words=add_stopwords)
  matrix, vocab = get_vector(month, count_vectorizer)
  lda = fit_lda(matrix, vocab)
  output = []
  for i in print_topics(lda, vocab):
    output.extend(i)
  return set(output)

In [121]:
gr['result'] = gr.kw_text.apply(lambda x: lda_month(x))

fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...
fitting lda...


#Запись результата в файл

In [127]:
hot_topic = gr[['result']]
hot_topic.reset_index(inplace=True)
hot_topic.columns = ['month', 'hot_topic']

#чтобы отсортировать по месяцам, переведем в формат даты
hot_topic['month'] = hot_topic.month.apply(lambda x: pd.to_datetime(x, format='%m.%Y'))
hot_topic = hot_topic.sort_values('month')
hot_topic['month'] = hot_topic.month.apply(lambda x: x.strftime('%m.%Y'))

hot_topic['hot_topic'] = hot_topic.hot_topic.apply(lambda x: ', '.join(x))

hot_topic.to_csv(os.path.join(folderpath, 'hot_topic.csv'), index=False)

In [139]:
for i in hot_topic[(hot_topic.month == '03.2019')|(hot_topic.month == '04.2020')].hot_topic:
  print(i)

боец, рубль, мать, ракета, погибнуть, сотрудник, регион, украина, слово, первыи, издание, самолет, ребенок, военныи, мир, фильм, москва, россиянин, президент, мочь, трамп, сторона
средство, клетка, область, маска, ссылаться, рука, заболевание, пользователь, первыи, представитель, соглашение, сделка, случаи, каждыи, последнии, мочь, коронавирус, вирус, тысяча, пандемия


Получилось довольно интересно, по крайней мере, видны основные направления. Апрель 2020 - разгар карантина в России, март 2019 - запуск ракеты Илона Маска, крушение самолета в Эфиопии, военные действия в Венесуэлле.

